# 4 - Feras Formidáveis

## 4.9 A senha de tamanho variável

### Introdução:

O objetivo desta atividade é resolver o problema da senha em algoritmos genéticos sem forneçer a informação
do tamanho da senha para a função que gera a população, considerando que a senha pode ser uma string de 1 até 30 caracteres. Boa parte do código foi adaptada dos materiais de aula do Daniel Cassar [1-X], em que criou-se scripts específicos para o projeto.

### Desenvolvimento:

Importando as bibliotecas necessárias [2-5]:

In [5]:
import random
from string import ascii_lowercase, ascii_uppercase, digits # importa as letras minúsculas, maiúsculas e dígitos
from Scripts import populacao_senha_size as cria_populacao
from Scripts import funcao_objetivo_pop_senha_size as funcao_objetivo
from Scripts import selecao_torneio_min as funcao_selecao
from Scripts import cruzamento_uniforme as funcao_cruzamento
from Scripts import mutacao_salto_tamanho as funcao_mutacao1
from Scripts import mutacao_simples as funcao_mutacao2
from Scripts import mutacao_salto as funcao_mutacao3

In [6]:
SENHA = list("Fazzio2020")
CARACTERES_POSSIVEIS = ascii_lowercase + ascii_uppercase + digits

TAMANHO_POPULACAO = 100
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO_LETRA = 0.025
CHANCE_DE_MUTACAO_TAMANHO = 0.05
TAMANHO_TORNEIO = 3

In [10]:
populacao = cria_populacao(TAMANHO_POPULACAO, CARACTERES_POSSIVEIS)

In [11]:
menor_fitness_geral = float("inf")
geracao = 0

while menor_fitness_geral != 0 and geracao <= 1000:

    # Seleção
    fitness = funcao_objetivo(populacao, SENHA)
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)

    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)

    # Mutação
    funcao_mutacao1(proxima_geracao, CHANCE_DE_MUTACAO_TAMANHO, list(CARACTERES_POSSIVEIS))
    funcao_mutacao2(proxima_geracao, CHANCE_DE_MUTACAO_LETRA, list(CARACTERES_POSSIVEIS))
    funcao_mutacao3(proxima_geracao, CHANCE_DE_MUTACAO_LETRA, list(CARACTERES_POSSIVEIS))

    # Encerramento
    populacao = proxima_geracao
    geracao += 1

    fitness = funcao_objetivo(populacao, SENHA)
    menor_fitness_observado = min(fitness)

    if menor_fitness_observado < menor_fitness_geral:
        menor_fitness_geral = menor_fitness_observado
        indice = fitness.index(menor_fitness_observado)
        candidato = populacao[indice]
        print(geracao, "".join(candidato))

1 kxeZesJK58
3 DxtoesBK5
5 NkeZesJK58
7 EacvnsA55
9 NaovnsBGD5
10 AeovnnIA55
16 Ehovnn8J75
18 EaovnsJ955
19 Eaovmv8555
22 Eaovnn8575
23 Eaovnn8555
24 Eaoven8555
27 Eaoveo8555
29 Eaovmo7555
31 Eaovmo7545
32 Faoveo7554
34 Eapvmo7544
37 Eapveo0545
39 Eapwfo0545
40 Eapwfo0544
42 Eaqvfo0543
44 Eaqwfo0543
47 Eaqwfo0541
50 Eaqwfo0441
53 Eaqwgo0540
56 Eaqwgo0440
57 Earwgo0440
59 Earxgo0440
60 Earwgo0040
71 Eawxgo1430
75 Eawygo1430
77 Eawxho2430
79 Eaxxho2430
81 Eaxyho2430
85 Faxyho2430
90 Eaxzio2430
93 Faxzio2430
95 Faxzio2330
99 Faxzio2030
103 Faxzio2020
115 Fayzio2020
123 Fazzio2020


### Conclusão:

Foi possível treinar uma rede neural em python puro para tarefas de classificação, embora ela não esteja bem otimizada, aumentando a perda ao longo do treinamento e prevendo apenas um rótulo para os dados do problema. Mesmo assim, foi interessante aprender sobre esse tipo de problema, aumentando meu conhecimento.

### Referências:

[1] CASSAR, Daniel. "ATP-303 NN 4.2 - Notebook MLP.ipynb". Material de Aula, 2025.

[2] Biblioteca Math. https://docs.python.org/3/library/math.html

[3] Biblioteca Random. https://docs.python.org/3/library/random.html

[4] Biblioteca Pandas. https://pandas.pydata.org/docs/user_guide/index.html#user-guide

[5] Biblioteca Seaborn. https://seaborn.pydata.org/

[6] Dataset estudado. https://archive.ics.uci.edu/dataset/53/iris

[7] Função de perda analisada. https://medium.com/ensina-ai/uma-explicação-visual-para-função-de-custo-binary-cross-entropy-ou-log-loss-eaee662c396c